In [6]:
import sys
import os

In [2]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [3]:
from importlib import reload

In [4]:
import pickle

In [68]:
import feature_extract
import build_graph

In [50]:
reload(build_graph)

<module 'build_graph' from '/home/immersinn/gits/rssfeed_link_collector/src/build_graph.py'>

In [11]:
bows_path = os.path.join(os.path.abspath(os.path.join('../..')), 'src/data/sample_bows.pkl')

In [17]:
with open(bows_path, 'rb') as f:
    bows = pickle.load(f)

In [69]:
cvs = feature_extract.CountVecSimple()

In [70]:
cvs.fit(bows)

In [71]:
bow_mat = cvs.transform(bows)

In [72]:
bow_mat.shape

(2075, 115)

In [62]:
import numpy
import pandas

In [64]:
pandas.Series(numpy.array(bow_mat.sum(axis=1)).reshape(bow_mat.shape[0],)).describe()

count    2075.000000
mean        2.685783
std         2.678095
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        53.000000
dtype: float64

In [73]:
import metrics

In [191]:
dds = metrics.calcJMSDocScores(bow_mat)

In [192]:
dds.shape

(2075, 2075)

### Simple JM Closeness Method

In [193]:
orig_index = cvs._bwm.lookup_docid('orig')

In [194]:
close_docs = numpy.argsort(dds[orig_index,:], )[-20:]
close_docs = [cvs._bwm.revlookup_docid(did) for did in close_docs]

In [207]:
reload(mysql_utils)

<module 'mysql_utils' from '/home/immersinn/gits/rssfeed_link_collector/src/mysql_utils.py'>

In [208]:
jms_close = mysql_utils.query_docsDetails(close_docs)

In [209]:
for t in jms_close.title:
    print(t)
    print('\n')

KLEIN – Trump-Russia Claims Rethought: How the Obama Administration Was Caught Politicizing Intelligence


CNN’s Acosta: Trump Only Calling on Conservative Media Can Only Be Described As ‘The Fix Is In’


NYT’s Friedman: Flynn Resignation Shows Russia Hacking Was On Scale With 9/11, Pearl Harbor


Dem Rep Moulton: If Trump Admin ‘Conspiring’ with Russia — That Is ‘The Definition of Treason’


Donald Trump Denounces ‘Un-American’ Intelligence Leaks to ‘Fake News’


Priebus: Take Trump Seriously When he Calls Media ‘The Enemy’


Hollywood Reporter’s Wolff to CNN’s Stelter on Trump Coverage: Media Becoming ‘Quite a Ridiculous Figure’


CNN’s Stelter: Flynn ‘A Big Moment for Investigative Journalism’


Law Enforcement Applauds Trump’s Attack on the ’Dishonest Media’


Media escalate Trump-Russia reporting as he rips 'fake news' conspiracies


Dems see disparity in handling of Clinton, Russia inquiries


First on CNN: John Kasich starts new political chapter


US officials corroborate aspec

### Simple Spectral 2-Way Method

In [106]:
import spectral_partition

In [107]:
reload(spectral_partition)

<module 'spectral_partition' from '/home/immersinn/gits/rssfeed_link_collector/src/spectral_partition.py'>

In [197]:
dds_sym = (dds + dds.T) / 2

In [198]:
C,M = spectral_partition.spectralGraphPartitionSingle(dds_sym, L=6.)

In [199]:
M

5000.9623860815318

In [171]:
orig_grp = numpy.where(C == C[orig_index])[0]
orig_grp = [cvs._bwm.revlookup_docid(did) for did in orig_grp]
_ = orig_grp.remove('orig')

In [172]:
len(orig_grp)

80

In [154]:
import mysql_utils

In [155]:
reload(mysql_utils)

<module 'mysql_utils' from '/home/immersinn/gits/rssfeed_link_collector/src/mysql_utils.py'>

In [211]:
doc_info = mysql_utils.query_docsDetails(orig_grp)

In [212]:
doc_info.head()

,link,title,summary,published
0,http://feedproxy.google.com/~r/breitbart/~3/6L...,"New York Times Accuses, then Exonerates Trump ...","<a href=""http://www.breitbart.com/big-governme...",2017-02-15 05:23:20
1,http://feedproxy.google.com/~r/breitbart/~3/Jc...,Cartel Robberies Continue Along Mexican Border...,"<a href=""http://www.breitbart.com/texas/2017/0...",2017-02-16 15:12:41
2,http://feedproxy.google.com/~r/breitbart/~3/S8...,"Industry Experts, Startup Companies Descend on...","<a href=""http://www.breitbart.com/tech/2017/02...",2017-02-16 16:23:12
3,http://feeds.foxnews.com/~r/foxnews/most-popul...,Report: Trump not fully briefed on exec order ...,President Trump was reportedly not fully brief...,2017-02-06 08:46:00
4,http://feeds.foxnews.com/~r/foxnews/most-popul...,Trump fully briefed on Flynn's calls with Russ...,President Trump was given a comprehensive summ...,2017-02-17 04:42:00


In [159]:
doc_info.summary[3]

'<a href="http://www.breitbart.com/sports/2017/02/09/video-charles-oakley-arrested-knicks-game-fighting-security-guard-going-knicks-owner/" title="VIDEO: Charles Oakley Arrested at Knicks Game, After Fighting Security Guard and Going After Knicks Owner"><img width="200" height="150" src="http://media.breitbart.com/media/2017/02/oakley-200x150.png" alt="screenshot" /></a><br /> Known as one of the “enforcers,” during his eighteen-year career in the NBA, Charles Oakley spent much of that enforcement time at Madison Square Garden. However, after Oakley engaged in some pushing, shoving, and highly questionable behavior at MSG on Wednesday night the only enforcement that occurred was law enforcement.<img src="http://feeds.feedburner.com/~r/breitbart/~4/FX7_PbkUWm0" height="1" width="1" alt=""/>'

In [176]:
doc_info.tail()

,link,title,summary,published
75,https://www.wired.com/video/what-s-inside-vodka,What's Inside - Vodka,"It’s the preferred poison of Russians, dieters...",2014-05-15 05:00:00
76,https://www.theguardian.com/environment/2017/f...,Green Investment Bank: Australian bidder woos ...,<p>Macquarie insists it is committed to renewa...,2017-02-23 10:37:54
77,http://www.bbc.co.uk/news/world-asia-39066658,Tiger knocks drone from the sky in keep-fit ex...,Tigers in China chase and knock a drone out of...,2017-02-23 13:27:23
78,http://www.cnn.com/2017/02/23/politics/fbi-ref...,FBI refused White House request to knock down ...,The FBI rejected a recent White House request ...,2017-02-23 23:02:58
79,http://feedproxy.google.com/~r/breitbart/~3/91...,The Nuclear Option — More Fake News: Media Fal...,"<a href=""http://www.breitbart.com/big-journali...",2017-02-23 23:50:57


In [177]:
doc_info.summary[77]

'Tigers in China chase and knock a drone out of the sky, in a keep fit exercise.'

In [144]:
other_grp = numpy.where(C != C[cvs._bwm.lookup_docid('orig')])[0]
other_grp = [cvs._bwm.revlookup_docid(did) for did in other_grp]

In [213]:
odoc_info = mysql_utils.query_docsDetails(other_grp)

In [214]:
odoc_info.head()

,link,title,summary,published
0,http://baltimore.cbslocal.com/2017/02/20/balti...,Surgeon makes boy new hand -- with his foot bones,"<div class=""feedflare"">\n<a href=""http://rss.c...",2017-02-21 23:04:36
1,http://bleacherreport.com/articles/2693359-ray...,MLBer drove for Uber in offseason,"<div class=""feedflare"">\n<a href=""http://rss.c...",2017-02-16 21:06:39
2,http://blogs.ft.com/the-world/2016/12/ft-podca...,FT podcast: World Weekly with Gideon Rachman,"<p><a href=""http://podcast.ft.com/2016/12/22/h...",2016-12-22 14:15:04
3,http://blogs.ft.com/the-world/2016/12/latam-vi...,LatAm Viva: Magic or realism?,"<p><strong><em>Sign up to receive <a href=""htt...",2016-12-17 11:30:44
4,http://blogs.ft.com/the-world/2017/01/ft-podca...,FT podcast: World Weekly with Gideon Rachman,"<p><a href=""http://podcast.ft.com/2017/01/04/t...",2017-01-05 16:35:53


In [215]:
odoc_info.tail()

,link,title,summary,published
1308,http://feeds.foxnews.com/~r/foxnews/national/~...,FBI using billboards for tips in killings of 2...,The FBI is using electronic billboards across ...,2017-02-23 21:36:00
1309,http://hosted.ap.org/dynamic/stories/A/ATH_RUS...,IAAF clears 3 more Russians to compete at worl...,Track and field authorities have cleared three...,2017-02-23 22:06:47
1310,http://feeds.foxnews.com/~r/foxnews/sports/~3/...,"Nutting: Pirates had 'step back' in '16, but t...","BRADENTON, Fla. (AP) The Pittsburgh Pirates mi...",2017-02-21 00:15:23
1311,http://feeds.foxnews.com/~r/foxnews/sports/~3/...,Jaguars release veteran defensive lineman Jare...,"JACKSONVILLE, Fla. (AP) The Jacksonville Jagua...",2017-02-20 22:21:13
1312,http://hosted.ap.org/dynamic/stories/W/WA_KIDN...,"Man wanted for kidnapping, shooting captured i...","FEDERAL WAY, Wash. (AP) -- Police say t...",2017-02-24 00:04:43
